In [1]:
!pip install lightrag[openai,groq,faiss-cpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 262.4 kB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


In [2]:
!pip install -U lightrag[openai,groq,faiss-cpu]
!pip show lightrag openai groq faiss-cpu

Name: lightrag
Version: 0.1.0b2
Summary: The Lightning Library for LLM Applications.
Home-page: https://github.com/SylphAI-Inc/LightRAG
Author: Li Yin
Author-email: li@sylphai.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: backoff, jinja2, jsonlines, numpy, python-dotenv, pyyaml, tiktoken, tqdm
Required-by: 
---
Name: openai
Version: 1.35.14
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
---
Name: groq
Version: 0.5.0
Summary: The official Python library for the groq API
Home-page: 
Author: 
Author-email: Groq <support@groq.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, typing-extensions
Required-by: 
---
Name: faiss-cpu
Version: 1.8.0.post1
Summary: A library for efficien

In [3]:
from getpass import getpass
import os


groq_api = getpass("Please enter your groq API : ")
openai_api = getpass("Please enter your openai API : ")


os.environ["OPENAI_API_KEY"] = openai_api
os.environ["GROQ_API_KEY"] = groq_api

print("API Keys has been set.")

Please enter your groq API : ··········
Please enter your openai API : ··········
API Keys has been set.


In [5]:
from dataclasses import dataclass, field
from typing import Dict

from lightrag.core import Component, Generator, DataClass, ModelClient
from lightrag.components.model_client import GroqAPIClient
from lightrag.components.output_parsers import JsonOutputParser

In [12]:
@dataclass
class QAOutput(DataClass):
  explanation : str = field(
      metadata = {"desc": "A brief explanation of the concept in one sentence."}
  )
  example : str = field(metadata={"desc": "An example of the concept in a sentence."})



qa_template = r"""<SYS>
you are a helpful asistant.
<OUTPUT_FORMAT>
{{output_format_str}}
</OUTPUT_FOMAT>
</SYS>
Uer: {{input_str}}
You:"""

In [22]:
# Create the task pipeline

class QA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: Dict):
        super().__init__()

        parser = JsonOutputParser(data_class=QAOutput, return_data_class=True)
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
            prompt_kwargs={"output_format_str": parser.format_instructions()},
            output_processors=parser,
        )

    def call(self, query: str):
        return self.generator.call({"input_str": query})

    async def acall(self, query: str):
        return await self.generator.acall({"input_str": query})

In [23]:
qa = QA(
    model_client = GroqAPIClient(),
    model_kwargs = {"model": "llama3-8b-8192"}
)

print(qa)

QA(
  (generator): Generator(
    model_kwargs={'model': 'llama3-8b-8192'}, 
    (prompt): Prompt(
      template: <SYS>
      you are a helpful asistant.
      <OUTPUT_FORMAT>
      {{output_format_str}}
      </OUTPUT_FOMAT>
      </SYS>
      Uer: {{input_str}}
      You:, prompt_kwargs: {'output_format_str': 'Your output should be formatted as a standard JSON instance with the following schema:\n```\n{\n    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",\n    "example": "An example of the concept in a sentence. (str) (required)"\n}\n```\n-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!\n-Use double quotes for the keys and string values.\n-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.\n-Follow the JSON formatting conventions.'}, prompt_variables: ['output_format_str', 'input_str']
    )
    (model_client): GroqAPI

In [24]:
qa("What is LLM?")

GeneratorOutput(data=QAOutput(explanation='LLM stands for Large Language Model, a type of artificial intelligence that is trained on a massive dataset of text and can generate human-like responses to natural language inputs.', example='For example, an LLM like language model can be used to generate chatbot responses, write articles, or even compose music.'), error=None, usage=None, raw_response='```\n{\n    "explanation": "LLM stands for Large Language Model, a type of artificial intelligence that is trained on a massive dataset of text and can generate human-like responses to natural language inputs.",\n    "example": "For example, an LLM like language model can be used to generate chatbot responses, write articles, or even compose music."\n}\n```', metadata=None)

In [25]:
qa.generator.print_prompt(
    output_format_str = qa.generator.output_processors.format_instructions(),
    input_str = "What is LLM?"
)

Prompt:
______________________
<SYS>
you are a helpful asistant.
<OUTPUT_FORMAT>
Your output should be formatted as a standard JSON instance with the following schema:
```
{
    "explanation": "A brief explanation of the concept in one sentence. (str) (required)",
    "example": "An example of the concept in a sentence. (str) (required)"
}
```
-Make sure to always enclose the JSON output in triple backticks (```). Please do not add anything other than valid JSON output!
-Use double quotes for the keys and string values.
-DO NOT mistaken the "properties" and "type" in the schema as the actual fields in the JSON output.
-Follow the JSON formatting conventions.
</OUTPUT_FOMAT>
</SYS>
Uer: What is LLM?
You:
